In [4]:
%matplotlib inline
import sys
import xarray as xr
import numpy as np
import os
import yaml
import math
from datetime import datetime, timedelta
from parcels import FieldSet, Field, VectorField, ParticleSet, JITParticle, ParcelsRandom, Variable

In [5]:
def get_timestamps(start,length):
    timestamps=[]
    duration = timedelta(days=length)
    for day in range(duration.days):
        timestamps.append([start + timedelta(days=day)])
    return np.array(timestamps, dtype='datetime64')

def find_temp(rootdir):
    dirs=[]
    for file in os.listdir(rootdir):
        d = os.path.join(rootdir, file)
        if os.path.isdir(d):
            dirs.append(d)
    temp=sorted(dirs, key=lambda x: os.path.getctime(x), reverse=True)[:1][0]
    return temp[-12:]

def newest(path):
    files = os.listdir(path)
    paths = [os.path.join(path, basename) for basename in files]
    return max(paths, key=os.path.getctime)


In [9]:
def path(local = 1):
    '''Change with your paths'''
    if local == 1:
        path = {'NEMO': '/Users/jvalenti/MOAD/data/',
        'coords': '/Users/jvalenti/MOAD/grid/coordinates_seagrid_SalishSea201702.nc',
        'mask': '/Users/jvalenti/MOAD/grid2/mesh_mask202108_TDV.nc',
        'out': '/Users/jvalenti/MOAD/results/',
        'home': '/Users/jvalenti/MOAD/analysis-jose/notebooks/parcels',
        'anim': '/Users/jvalenti/MOAD/animations'}
    else:
        path = {'NEMO': '/results2/SalishSea/nowcast-green.202111/',
        'coords': '/ocean/jvalenti/MOAD/grid/coordinates_seagrid_SalishSea201702.nc',
        'coordsWW3': '/ocean/jvalenti/MOAD/grid2/WW3_grid.nc',
        'mask': '/ocean/jvalenti/MOAD/grid2/mesh_mask202108_TDV.nc',
        'bat': '/ocean/jvalenti/MOAD/grid/bathymetry_202108.nc',
        'out': '/home/jvalenti/MOAD/results',
        'home': '/home/jvalenti/MOAD/analysis-jose/notebooks/parcels',
        'anim': '/home/jvalenti/MOAD/animations'}
    return path

def load_config(config_yaml):
   with open(config_yaml[0]) as f:
       config = yaml.safe_load(f)
   return config

In [10]:
# Define paths
local=0
paths = path(local)
#Load configuration from .yaml
param = load_config(['/home/jvalenti/MOAD/analysis-jose/OParcels/yaml/test.yaml'])
#Definitions
start = datetime(param['startdate']['year'], param['startdate']['month'], param['startdate']['day']) #Start date
length = param['param']['length'] # Set Time length [days] 
dt = param['param']['dt'] #toggle between - or + to pick backwards or forwards 
N0 = param['param']['N'] # number of deploying locations
n = param['param']['n'] # 1000   # number of particles per location
dmin = param['param']['dmin'] #minimum depth
dd = param['param']['dd'] #max depth difference from dmin
name = param['file']['name'] #name output file
dtp = param['param']['dtp'] #how often particle released in hours
odt = param['param']['odt'] #how often data is recorded
rrr = param['param']['r'] #radious of particle deployment
distr = param['param']['distr']
MFc = param['param']['MFc']

In [22]:
def p_deploy(N,n,dmin,dd,r = 1000):
    #r is radius of particle cloud [m]
    deg2m = 111000 * np.cos(50 * np.pi / 180)
    var = (r / (deg2m * 3))**2
    x_offset, y_offset = np.random.multivariate_normal([0, 0], [[var, 0], [0, var]], [n,N]).T
    if isinstance(dmin,int):
        zvals1 = dmin + np.random.random_sample([n,N]).T*(dd)
    else:
        zvals = []
        zvals1 = []
        for dept in dmin:
            zvals.append(dept + np.random.random_sample([n]).T*(dd))
        for i in range(len(zvals)):   
            zvals1=np.concatenate((zvals1[:],zvals[i]))
    return x_offset, y_offset, zvals1


def make_prefix(date, path, res='h'):
    """Construct path prefix for local SalishSeaCast results given date object and paths dict
    e.g., /results2/SalishSea/nowcast-green.201905/daymonthyear/SalishSea_1h_yyyymmdd_yyyymmdd
    """

    datestr = '_'.join(np.repeat(date.strftime('%Y%m%d'), 2))
    folder = date.strftime("%d%b%y").lower()
    prefix = os.path.join(path, f'{folder}/SalishSea_1{res}_{datestr}')
    
    return prefix

def get_WW3_path(date):
    """Construct WW3 results path given the date
    e.g., /opp/wwatch3/nowcast/SoG_ww3_fields_YYYYMMDD_YYYYMMDD.nc
    :arg date: date of WW3 record
    :type date: :py:class:`datetime.datetime`
    :returns: WW3 path
    :rtype: str
    """
    # Make WW3 path
    path = '/opp/wwatch3/hindcast'
    path2 = '/opp/wwatch3/nowcast'
    datestr = [date.strftime(fmt) for fmt in ('%d%b%y', '%Y%m%d_%Y%m%d')]
    path = os.path.join(path, datestr[0].lower(), f'SoG_ww3_fields_{datestr[1]}.nc')
    if not os.path.exists(path):
        path = os.path.join(path2, datestr[0].lower(), f'SoG_ww3_fields_{datestr[1]}.nc')
        if not os.path.exists(path):    
            raise ValueError(f"No WW3 record found for the specified date {date.strftime('%Y-%b-%d')}")

    return path

def get_Fraser_path(date):
    """Construct Fraser river outflow path given the date
    e.g., /results/forcing/rivers/#R201702DFraCElse_yYYYYmMMdDD.nc
    :arg date: date of fraser outflow record 
    :type date: :py:class:`datetime.datetime`
    :returns: nc path
    :rtype: str
    """

    # Make Fraser path
    path = '/results/forcing/rivers/'
    datestr = [date.strftime(fmt) for fmt in ('%d%b%y', 'y%Ym%md%d')]
    path = os.path.join(path, f'R201702DFraCElse_{datestr[1]}.nc')
    if not os.path.exists(path):
        raise ValueError(f"No file found for the specified date {date.strftime('%Y-%b-%d')}")
    return path

In [13]:
#Set deploy coordinates following yaml   

clat = param['param']['lats']
clon = param['param']['lons']
#clon, clat = [float(outf_lon)],[float(outf_lat)] 
duration = timedelta(days=length)
#Set deploy locations
if distr == 'hmg':
    clat,clon = p_unidist(N0,N0)
    N = len(clat)
elif distr == 'trst':
    clat,clon = transect_deploy(clat,clon,N0)
    N = N0
elif distr == 'std':
    N = len(param['param']['lats'])
elif distr == 'pd':
    clat, clon, N = pandas_deploy(N0,MFc,int(dtp))
    n = 1

x_offset, y_offset, z = p_deploy(N,n,dmin,dd,rrr)

lon = np.zeros([N,n])
lat = np.zeros([N,n])
for i in range(N):
    lon[i,:]=(clon[i] + x_offset[i,:])
    lat[i,:]=(clat[i] + y_offset[i,:])

In [14]:
#Set start date time and the name of the output file

daterange = [start+timedelta(days=i) for i in range(length)]
fn =  name + '_'.join(d.strftime('%Y%m%d')+'_1n' for d in [start, start+duration]) + '.nc'
outfile = os.path.join(paths['out'], fn)

In [20]:
def filename_set(start,length,varlist=['U','V','W'],local=0):
    '''filename,variables,dimensions = filename_set(start,duration,varlist=['U','V','W'],local=1)
    Modify function to include more default variables
    define start as: e.g, datetime(2018, 1, 17)
    length= number of days'''
    
    duration = timedelta(days=length)
    #Build filenames
    paths = path(local)
    Rlist,Tlist,Ulist, Vlist, Wlist = [], [], [], [], []
    Waveslist = []
    Flist = []
    Biolist,MZlist = [],[]
   
    for day in range(duration.days):
        path_NEMO = make_prefix(start + timedelta(days=day), paths['NEMO'])
        path_NEMO_d = make_prefix(start + timedelta(days=day), paths['NEMO'],res='d')
        Ulist.append(path_NEMO + '_grid_U.nc')
        Vlist.append(path_NEMO + '_grid_V.nc')
        Wlist.append(path_NEMO + '_grid_W.nc')
        Tlist.append(path_NEMO + '_grid_T.nc')
        Biolist.append(path_NEMO_d + '_prod_T.nc')
        Waveslist.append(get_WW3_path(start + timedelta(days=day)))
        Flist.append(get_Fraser_path(start + timedelta(days=day)))
        

    # Load NEMO forcing 
    filenames = {
        'U': {'lon': paths['coords'], 'lat': paths['coords'], 'depth': Ulist, 'data': Ulist},
        'V': {'lon': paths['coords'], 'lat': paths['coords'], 'depth': Ulist, 'data': Vlist},
        'W': {'lon': paths['coords'], 'lat': paths['coords'], 'depth': Wlist[0], 'data': Wlist},
        'Kz': {'lon': paths['coords'], 'lat': paths['coords'], 'depth': Wlist[0], 'data': Wlist},
        'T': {'lon': paths['coords'], 'lat': paths['coords'], 'depth': Tlist[0], 'data': Tlist},
        'S': {'lon': paths['coords'], 'lat': paths['coords'], 'depth': Tlist[0], 'data': Tlist},
        'ssh': {'lon': paths['coords'], 'lat': paths['coords'], 'data': Tlist},
        'R': {'lon': paths['coords'], 'lat': paths['coords'], 'depth': Tlist[0], 'data': Tlist},
        'Bathy' : {'lon': paths['coords'], 'lat': paths['coords'], 'data': paths['bat']},
        'gdepth' : {'lon': paths['coords'], 'lat': paths['coords'],'depth': Ulist, 'data': paths['mask']},
        'totdepth' : {'lon': paths['coords'], 'lat': paths['coords'], 'data': paths['mask']},
        'US' : {'lon': paths['coordsWW3'], 'lat': paths['coordsWW3'], 'data': Waveslist},
        'VS' : {'lon': paths['coordsWW3'], 'lat': paths['coordsWW3'], 'data': Waveslist},
        'WL' : {'lon': paths['coordsWW3'], 'lat': paths['coordsWW3'], 'data': Waveslist},
        'FS' :  {'lon': paths['coords'], 'lat': paths['coords'],'data': Flist},
        'Diat' : {'lon': paths['coords'], 'lat': paths['coords'], 'depth': Tlist[0], 'data': Biolist},
        'Flag' : {'lon': paths['coords'], 'lat': paths['coords'], 'depth': Tlist[0], 'data': Biolist},
        'Vol' : {'lon': paths['coords'], 'lat': paths['coords'], 'depth': Wlist[0],'data': paths['mask']}
    }
    variables = {'U': 'vozocrtx', 'V': 'vomecrty','W': 'vovecrtz','gdepth':"depthu",'T':'votemper','S':'vosaline','R':'sigma_theta',
        'US':'uuss','VS':'vuss','WL':'lm','Bathy':'Bathymetry','FS':'rorunoff','Kz':'vert_eddy_diff',
        'MZ':'microzooplankton','Diat':'PPDIATNO3','Flag':'PPPHYNO3','ssh':'sossheig','totdepth':'totaldepth','Vol':'volume'}
        
    file2,var2 = {},{}
    for var in varlist:
        file2[var]=filenames[var]
        var2[var]=variables[var]
    return file2,var2

In [23]:
varlist=['U','V','gdepth']
filenames,variables=filename_set(start,length,varlist,local)

dimensions = {
    "U": {"lon": "glamf", "lat": "gphif", "depth": "not_yet_set", "time": "time_counter"},
    "V": {"lon": "glamf", "lat": "gphif", "depth": "not_yet_set", "time": "time_counter"},
    "depth_u": {"lon": "glamf", "lat": "gphif", "depth": "not_yet_set", "time": "time_counter"},
}
variables = {
    "U": "vozocrtx",
    "V": "vomecrty",
    "gdepth": "depthu",
}

In [24]:
fieldset = FieldSet.from_netcdf(
    filenames, variables, dimensions, mesh="flat", allow_time_extrapolation=True
)
fieldset.U.set_depth_from_field(fieldset.depth_u)
fieldset.V.set_depth_from_field(fieldset.depth_u)

NotImplementedError: Vertically adaptive meshes not implemented for from_netcdf()

In [ ]:
####BUILD FIELDS FOR SIMULATION######
#Fill in the list of variables that you want to use as fields
varlist=['U','V','W']
filenames,variables=filename_set(start,length,varlist,local)
dimensions = {'lon': 'glamf', 'lat': 'gphif', 'depth': 'depthw','time': 'time_counter'}
field_set=FieldSet.from_nemo(filenames, variables, dimensions, allow_time_extrapolation=True)

####BUILD Particle typeN######
MPParticle = particle_maker(param)

In [ ]:
if restart==1:
    name_temp=find_temp(paths['out'])
    os.system(f"cd {paths['out']} && parcels_convert_npydir_to_netcdf {name_temp}")
    outfile=newest(paths['out'])
    pset = ParticleSet.from_particlefile(field_set, MPParticle,outfile)
else:
    if dtp == 0:
        pset = ParticleSet.from_list(field_set, MPParticle, lon=lon, lat=lat, depth=z,time=start+timedelta(hours=odt))
    else:
        pset = ParticleSet.from_list(field_set, MPParticle, lon=lon, lat=lat, depth=z, repeatdt = timedelta(hours=dtp))

In [ ]:
def kernel_asem(pset,config):
    KER = AdvectionRK4_3D 
    if 'Buoyancy' in config['kernel']:
        KER += pset.Kernel(Buoyancy)
    if 'Stokes_drift' in config['kernel']:
        KER += pset.Kernel(Stokes_drift)
    if 'Beaching' in config['kernel']:
        KER += pset.Kernel(Beaching)
        KER += pset.Kernel(Unbeaching)
    if 'Turb_mix' in config['kernel']:
        KER += pset.Kernel(turb_mix)
    if 'Biofilm' in config['kernel']:
        KER += pset.Kernel(Biofilm)
    if 'Stokes_driftRK4_3D' in config['kernel']:
        KER += pset.Kernel(Stokes_driftRK4_3D)
    
    return KER

In [ ]:
KERNELS = kernel_asem(pset,param)
pset.execute(KERNELS,
            runtime=duration, 
            dt=dt,
            output_file=pset.ParticleFile(name=outfile, outputdt=timedelta(hours=odt)),
            recovery={ErrorCode.ErrorOutOfBounds: DeleteParticle})

INFO: Compiled ArrayMPParticleAdvectionRK4_3DStokes_driftBeachingUnbeachingturb_mix ==> /tmp/parcels-2894/libb390fad00b418f4b4ce3ef7e2edef32f_0.so
INFO: Temporary output files are stored in /home/jvalenti/MOAD/results/out-RKAZBHDF.
INFO: You can use "parcels_convert_npydir_to_netcdf /home/jvalenti/MOAD/results/out-RKAZBHDF" to convert these to a NetCDF file during the run.
  8%|▊         | 21600.0/259200.0 [00:06<01:31, 2606.43it/s] 

KeyboardInterrupt: 

  8%|▊         | 21600.0/259200.0 [00:19<01:31, 2606.43it/s]